In [1]:
import sqlite3
import pandas as pd
import numpy as np
from typing import Dict, Any
from datetime import timedelta

In [2]:
def connect_to_db(db_path: str) -> sqlite3.Connection:
    """Efficiently connect to SQLite database"""
    return sqlite3.connect(db_path, detect_types=sqlite3.PARSE_DECLTYPES)

def preprocess_datetime(df: pd.DataFrame) -> pd.DataFrame:
    """Central datetime preprocessing to reduce redundant operations"""
    datetime_columns = ['创建时间', '预定日期', '下单时间']
    for col in datetime_columns:
        if col in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                except Exception as e:
                    print(f"转换 {col} 列时出错：{e}")
    return df

In [3]:
def get_db_connection():
    conn = sqlite3.connect('db/ideapod.db')
    conn.row_factory = sqlite3.Row
    return conn


In [7]:
def analyze(conn):
    """主分析函数"""
    catering_df = pd.read_sql_query("SELECT * FROM Catering", conn)

    catering_df = preprocess_datetime(catering_df)
    user_df= catering_df[catering_df['商品'].str.contains('图书馆')]

    target_date = pd.to_datetime('2025-01-09')
    filtered_df = user_df[user_df['下单时间'].dt.date == target_date.date()]

    return filtered_df

print (analyze(get_db_connection()))

       会员号                下单时间 预约单预约时间        对账日期  \
36525  nan 2025-01-09 17:12:00    None  2025-01-09   

                                订单号  \
36525  3479125010917120043550050020   

                                                    商品 支付方式    金额    净额    实收  \
36525  个人「图书馆专注」3/6小时套餐（最高可省15元）x1.000,「图书馆」x 1hx1.000   微信  55.0  47.5  47.5   

       ...  赠送  开票状态  服务方式  订单来源  原订单号  退单类型   下单人    备注   佣金    集点  
36525  ...   0  None    堂食  门店点单  None  None  Dora  None  0.0  None  

[1 rows x 23 columns]
